In [1]:
import os
import json
from PIL import Image
import torch
from torchvision import transforms
import torch
from transformers import BertModel, BertTokenizer
import clip
from PIL import Image
image_folder = 'C:\\Users\\wjw\\Desktop\\111\\hateful_memes\\img\\'  # 图像文件夹路径
json_file = 'test_seen.jsonl'  # JSON文件路径列表
# 存储所有图像、文本和标签
images = []
texts = []
labels = []
image_paths = []
Id=[]
# 遍历每个JSON文件
with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f]
        for entry in data:
            img_name = entry['img']  # 获取图像路径
            img_path = img_name.replace('img/','img//')
            iid=img_name.replace('png','').replace('img/','')
            if os.path.isfile(img_path):
                try:
                    img = Image.open(img_path).convert('RGB')  # 确保图像为RGB格式
                    images.append(img)
                    texts.append(entry['text'])                # 存储对应文本
                    labels.append(entry['label'])               # 存储标签
                    image_paths.append(img_path)
                    Id.append(iid)
                except Exception as e:
                    print(f'Error loading image {img_path}: {e}')

In [2]:
import torch
from torchvision import transforms
from transformers import AutoFeatureExtractor, AutoModel
from transformers import ViTModel, ViTFeatureExtractor

feature_extractor1 = ViTFeatureExtractor.from_pretrained("WinKawaks/vit-small-patch16-224")
model1 = ViTModel.from_pretrained("WinKawaks/vit-small-patch16-224")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs1 = feature_extractor1(images=images, return_tensors="pt")
model1.to(device)
model1.eval()
with torch.no_grad():
   outputs1 = model1(**inputs1)
   last_hidden_state1 = outputs1.last_hidden_state
print(last_hidden_state1.shape)# 获取特征表示

C:\Users\wjw\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-small-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([815, 197, 384])


In [3]:
from transformers import BertModel, BertTokenizer

# 加载预训练的BERT模型和分词器，这里以'bert-base-uncased'为例
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 将模型设置为评估模式，这一步很关键，它会关闭一些在训练时才需要的操作，比如Dropout等
model.eval()
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

In [4]:
with torch.no_grad():  # 禁用梯度计算，因为只是获取特征，不需要进行反向传播和参数更新
    outputs1 = model(**encoded_inputs)
    text_features = outputs1.last_hidden_state

In [280]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

In [281]:
x=last_hidden_state1.reshape(last_hidden_state1.size(0), -1)

In [282]:
x1=text_features.reshape(text_features.size(0), -1)

In [283]:
x.shape

torch.Size([815, 75648])

In [284]:
x1.shape

torch.Size([815, 56832])

In [285]:
import torch
import torch.nn as nn

class MultipleLinearTransform(nn.Module):
    def __init__(self, input_dim, num_transforms):
        super(MultipleLinearTransform, self).__init__()
        # 定义线性层
        self.linear = nn.Linear(input_dim, input_dim)
        # 定义变换的次数
        self.num_transforms = num_transforms

    def forward(self, x):
        # 初始化累加结果
        total_result = torch.zeros_like(x)
        current_input = x
        for _ in range(self.num_transforms):
            # 进行线性变换
            transformed = self.linear(current_input)
            # 累加结果
            total_result += transformed
            # 更新当前输入
            current_input = transformed
        return total_result


class DualNetwork(nn.Module):
    def __init__(self, features_dim1, features_dim2, inflect_dim, num_transforms):
        super(DualNetwork, self).__init__()
        # 定义两个单层网络
        self.model1 = nn.Linear(features_dim1, inflect_dim)  # 第一个网络
        self.model2 = nn.Linear(features_dim2, inflect_dim)  # 第二个网络
        self.transform = MultipleLinearTransform(inflect_dim, num_transforms)

    def forward(self, x1, x2):
        # 前向传播
        output1 = self.model1(x1)
        output2 = self.model2(x2)

        # 使用 MultipleLinearTransform 进行多次线性变换并求和
        sum_result1 = self.transform(output1)
        sum_result2 = self.transform(output2)

        # 标准化输出
        standardized_output1 = self.standardize(sum_result1)
        standardized_output2 = self.standardize(sum_result2)

        # 计算标准化后的点积
        dot_product = torch.matmul(standardized_output1, standardized_output2.T)
        return dot_product,sum_result1,sum_result2

    def standardize(self, tensor):
        # 标准化函数
        mean = tensor.mean(dim=0, keepdim=True)
        std = tensor.std(dim=0, keepdim=True)
        std = torch.clamp(std, min=1e-8)  # 避免除零错误
        return (tensor - mean) / std

In [286]:
import torch

torch.serialization.add_safe_globals([DualNetwork])
model = torch.load('entire_simple_model2.pth', weights_only=False)

# 将模型设置为评估模式（如果是用于推理）
model.eval()

# 进行推理
with torch.no_grad():
    output = model(x,x1)
    print("模型推理结果:", output)

模型推理结果: (tensor([[ 4.5134e+02,  3.1725e+02, -6.4310e+01,  ...,  8.9610e+00,
          1.7261e+02, -1.5789e+02],
        [ 1.9184e+02,  2.5513e+02,  4.1967e+00,  ..., -1.1964e+01,
          3.3775e+01, -5.6225e+01],
        [-2.6910e+01,  5.5510e+01,  3.7817e+02,  ...,  1.8870e+02,
         -1.7032e-01, -3.8109e+01],
        ...,
        [ 3.8513e+01,  5.4067e+01,  2.1033e+02,  ...,  3.1428e+02,
          1.3186e+02, -1.1612e+01],
        [ 1.4617e+02,  1.2648e+02, -2.1619e+01,  ...,  1.0221e+02,
          5.0787e+02, -2.1435e+02],
        [-1.3078e+02, -1.2651e+02, -7.1087e+01,  ..., -4.6604e+01,
         -1.8374e+02,  3.0316e+02]]), tensor([[-27.4376,  10.6988,  -7.3310,  ..., -37.6511,  47.4127,  35.0639],
        [-37.0884,  23.0469, -15.5403,  ..., -37.8338,  36.8927,  37.2083],
        [-42.8019,  20.4881, -27.5798,  ..., -46.1525,  39.5579,  31.0815],
        ...,
        [-28.3854,  16.7170, -14.3914,  ..., -41.6562,  45.1141,  28.6936],
        [-32.9315,  22.4189, -14.6953,  .

In [287]:
output[1].shape

torch.Size([815, 512])

In [288]:
x_combine=torch.cat((output[1], output[2]), dim=1)

In [289]:
features_np = x_combine.detach().numpy()

In [290]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(features_np, labels, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [291]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
     def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # 第一个全连接层
        self.fc2 = nn.Linear(256, 128)           # 第二个全连接层
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)# 输出层
        self.relu = nn.ReLU()                   # ReLU 激活函数
        self.sigmoid = nn.Sigmoid()             # Sigmoid 激活函数用于二分类

     def forward(self, x):
        x = self.relu(self.fc1(x))              # 第一层
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))           # 输出层
        return x   
input_size = 1024  # 输入特征的维度
model3 = BinaryClassifier(input_size)

# 打印模型结构
print(model3)

BinaryClassifier(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [292]:
import numpy as np
import random
criterion = nn.BCELoss()
optimizer = optim.Adam(model3.parameters(), lr=0.001)
num_epochs = 30
acc=[]
set_seed(42)
for epoch in range(num_epochs):
    model3.train()  # 设置为训练模式
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model3(X_train)
    loss = criterion(outputs, y_train.view(-1, 1).float())
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    model3.eval()  # 设置为评估模式
    with torch.no_grad():  # 不计算梯度
        val_outputs = model3(X_val)  # 在验证集上进行前向传播
        val_predicted = (val_outputs >= 0.5).float()  # 将输出转换为0或1的预测
        correct_predictions = (val_predicted.view(-1) == y_val).sum().item()  # 计算正确预测的数量
        val_accuracy = correct_predictions / y_val.size(0)  # 计算验证集的准确率
        acc.append(val_accuracy)
    #if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

Epoch [1/30], Loss: 0.7087, Val Accuracy: 0.4553
Epoch [2/30], Loss: 0.6945, Val Accuracy: 0.5203
Epoch [3/30], Loss: 0.7311, Val Accuracy: 0.4797
Epoch [4/30], Loss: 0.6946, Val Accuracy: 0.4959
Epoch [5/30], Loss: 0.6935, Val Accuracy: 0.4797
Epoch [6/30], Loss: 0.6992, Val Accuracy: 0.5203
Epoch [7/30], Loss: 0.7078, Val Accuracy: 0.4797
Epoch [8/30], Loss: 0.6900, Val Accuracy: 0.4797
Epoch [9/30], Loss: 0.7011, Val Accuracy: 0.5447
Epoch [10/30], Loss: 0.6886, Val Accuracy: 0.5203
Epoch [11/30], Loss: 0.6977, Val Accuracy: 0.4797
Epoch [12/30], Loss: 0.7002, Val Accuracy: 0.4797
Epoch [13/30], Loss: 0.6888, Val Accuracy: 0.5203
Epoch [14/30], Loss: 0.6955, Val Accuracy: 0.5285
Epoch [15/30], Loss: 0.6871, Val Accuracy: 0.4878
Epoch [16/30], Loss: 0.6864, Val Accuracy: 0.5122
Epoch [17/30], Loss: 0.6825, Val Accuracy: 0.5447
Epoch [18/30], Loss: 0.6849, Val Accuracy: 0.5203
Epoch [19/30], Loss: 0.6805, Val Accuracy: 0.5122
Epoch [20/30], Loss: 0.6831, Val Accuracy: 0.5203
Epoch [21

In [293]:
from sklearn.metrics import f1_score

# 将模型设置为评估模式（如果是用于推理）
model3.eval()
with torch.no_grad():
    val_outputs = model3(X_val)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 50.41%
Validation F1 Score: 0.4696


In [294]:
# 将模型设置为评估模式（如果是用于推理）
model3.eval()
with torch.no_grad():
    val_outputs = model3(X_test)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 53.92%
Validation F1 Score: 0.4198


In [295]:
import torch

def add_gaussian_noise_pytorch(features, mean=0, std=0.1):
    """
    给 PyTorch 张量形式的特征添加高斯噪声
    :param features: 处理好的特征张量
    :param mean: 高斯噪声的均值
    :param std: 高斯噪声的标准差
    :return: 添加噪声后的特征张量
    """
    # 生成与特征张量形状相同的高斯噪声
    noise = torch.randn(features.size()) * std + mean
    # 将噪声添加到特征上
    noisy_features = features + noise
    return noisy_features
# 添加高斯噪声
X_test_noise = add_gaussian_noise_pytorch(X_test, mean=0, std=0.1)
y_test_noise=y_test+y_test
print("Original features shape:", X_test_noise.shape)
print("Noisy features shape:", y_test_noise.shape)

Original features shape: torch.Size([204, 1024])
Noisy features shape: torch.Size([204])


In [296]:
from sklearn.metrics import f1_score
model.eval()  # 设置为评估模式
with torch.no_grad():
    val_outputs = model3(X_test_noise)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test_noise).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 37.75%
Validation F1 Score: 0.4224


In [297]:
X_val_noise = add_gaussian_noise_pytorch(X_val, mean=0, std=0.1)
y_val_noise=y_val+y_val

In [298]:
from sklearn.metrics import f1_score
model.eval()  # 设置为评估模式
with torch.no_grad():
    val_outputs = model3(X_val_noise)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val_noise).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 28.46%
Validation F1 Score: 0.4696


In [299]:
import torch
import numpy as np

def add_salt_and_pepper_noise(features, salt_prob=0.01, pepper_prob=0.01):
    """
    给 PyTorch 张量形式的特征添加椒盐噪声
    :param features: 处理好的特征张量
    :param salt_prob: 添加盐噪声的概率
    :param pepper_prob: 添加胡椒噪声的概率
    :return: 添加噪声后的特征张量
    """
    # 创建一个与特征相同形状的噪声张量
    noisy_features = features.clone()
    
    # 生成随机数
    random_values = torch.rand(features.size())
    
    # 添加盐噪声（随机选择一些像素点设置为最大值）
    noisy_features[random_values < salt_prob] = 1.0  # 1.0 表示盐噪声
    
    # 添加胡椒噪声（随机选择一些像素点设置为最小值）
    noisy_features[random_values > (1 - pepper_prob)] = 0.0  # 0.0 表示胡椒噪声
    
    return noisy_features

# 添加椒盐噪声
X_test_noise = add_salt_and_pepper_noise(X_test, salt_prob=0.01, pepper_prob=0.01)
y_test_noise = y_test + y_test  # 根据需要进行修改
print("Original features shape:", X_test.shape)
print("Noisy features shape:", X_test_noise.shape)

Original features shape: torch.Size([204, 1024])
Noisy features shape: torch.Size([204, 1024])


In [300]:
from sklearn.metrics import f1_score
model.eval()  # 设置为评估模式
with torch.no_grad():
    val_outputs = model3(X_test_noise)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test_noise).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 36.27%
Validation F1 Score: 0.4146


In [301]:
X_val_noise = add_salt_and_pepper_noise(X_val, salt_prob=0.01, pepper_prob=0.01)
y_val_noise=y_val+y_val
from sklearn.metrics import f1_score
model.eval()  # 设置为评估模式
with torch.no_grad():
    val_outputs = model3(X_val_noise)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val_noise).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 28.46%
Validation F1 Score: 0.4696


In [302]:
import torch.nn as nn

class FeatureMapper(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(FeatureMapper, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

In [303]:
def attention(query, key, value):
        scores = torch.matmul(query, key.transpose(-2, -1))  # 点积
        weights = nn.functional.softmax(scores, dim=-1)  # softmax
        return torch.matmul(weights, value)  # 加权求和

In [304]:
def fusion(original, attended):
        return original + attended  # 或者使用 torch.cat 进行连接

In [305]:
x1.shape

torch.Size([815, 56832])

In [306]:
image_mapper = FeatureMapper(75648, 512)
text_mapper = FeatureMapper(56832, 512)

mapped_image = image_mapper(x)
mapped_text = text_mapper(x1)

# 计算注意力
attended_image = attention(mapped_text, mapped_image, mapped_image)
attended_text = attention(mapped_image, mapped_text, mapped_text)

# 特征融合
fused_image = fusion(mapped_image, attended_image)
fused_text = fusion(mapped_text, attended_text)

In [307]:
x_fuse=torch.cat((fused_image, fused_text), dim=1)
x_fuse = x_fuse.detach().numpy()
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(x_fuse, labels, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [308]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
     def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # 第一个全连接层
        self.fc2 = nn.Linear(128, 64)           # 第二个全连接层
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)# 输出层
        self.relu = nn.ReLU()                   # ReLU 激活函数
        self.sigmoid = nn.Sigmoid()             # Sigmoid 激活函数用于二分类

     def forward(self, x):
        x = self.relu(self.fc1(x))              # 第一层
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))           # 输出层
        return x                                 # 返回概率值

# 使用示例
input_size = 1024  # 输入特征的维度
model = BinaryClassifier(input_size)

# 打印模型结构
print(model)

BinaryClassifier(
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [309]:
import numpy as np
import random
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30
acc1=[]
for epoch in range(num_epochs):
    model.train()  # 设置为训练模式
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1).float())
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    model.eval()  # 设置为评估模式
    with torch.no_grad():  # 不计算梯度
        val_outputs = model(X_val)  # 在验证集上进行前向传播
        val_predicted = (val_outputs >= 0.5).float()  # 将输出转换为0或1的预测
        correct_predictions = (val_predicted.view(-1) == y_val).sum().item()  # 计算正确预测的数量
        val_accuracy = correct_predictions / y_val.size(0)  # 计算验证集的准确率
        acc1.append(val_accuracy)
    #if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

Epoch [1/30], Loss: 0.6932, Val Accuracy: 0.4878
Epoch [2/30], Loss: 0.6913, Val Accuracy: 0.5285
Epoch [3/30], Loss: 0.6893, Val Accuracy: 0.5285
Epoch [4/30], Loss: 0.6867, Val Accuracy: 0.5203
Epoch [5/30], Loss: 0.6836, Val Accuracy: 0.5528
Epoch [6/30], Loss: 0.6797, Val Accuracy: 0.5610
Epoch [7/30], Loss: 0.6752, Val Accuracy: 0.5528
Epoch [8/30], Loss: 0.6698, Val Accuracy: 0.5610
Epoch [9/30], Loss: 0.6635, Val Accuracy: 0.5366
Epoch [10/30], Loss: 0.6556, Val Accuracy: 0.5285
Epoch [11/30], Loss: 0.6466, Val Accuracy: 0.5203
Epoch [12/30], Loss: 0.6368, Val Accuracy: 0.5122
Epoch [13/30], Loss: 0.6255, Val Accuracy: 0.5041
Epoch [14/30], Loss: 0.6125, Val Accuracy: 0.5285
Epoch [15/30], Loss: 0.5973, Val Accuracy: 0.5203
Epoch [16/30], Loss: 0.5796, Val Accuracy: 0.5203
Epoch [17/30], Loss: 0.5600, Val Accuracy: 0.4878
Epoch [18/30], Loss: 0.5403, Val Accuracy: 0.5122
Epoch [19/30], Loss: 0.5209, Val Accuracy: 0.5041
Epoch [20/30], Loss: 0.4997, Val Accuracy: 0.5041
Epoch [21

In [310]:
from sklearn.metrics import f1_score
with torch.no_grad():
    val_outputs = model(X_val)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 48.78%
Validation F1 Score: 0.5116


In [311]:
with torch.no_grad():
    val_outputs = model(X_test)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 52.94%
Validation F1 Score: 0.4894


In [312]:
len(acc)

30

In [313]:
import torch
import torch.nn as nn

class FeatureFusion(nn.Module):
    def __init__(self, input_dim_u1, input_dim_u2):
        super(FeatureFusion, self).__init__()
        # 定义线性层以调整特征维度
        self.linear_u1 = nn.Linear(input_dim_u1, 512)
        self.linear_u2 = nn.Linear(input_dim_u2, 512)

    def forward(self, u1, u2):
        # 调整特征维度
        u1_transformed = self.linear_u1(u1)
        u2_transformed = self.linear_u2(u2)

        # 计算外积
        # 相加
        z = u1_transformed + u2_transformed
        return z

# 示例输入
feature_dim_u1 = 75648  # 模态1特征维度
feature_dim_u2 = 56832  # 模态2特征维度
feature_fusion = FeatureFusion(feature_dim_u1, feature_dim_u2)

# 计算融合特征
z = feature_fusion(x, x1)

In [314]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
     def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # 第一个全连接层
        self.fc2 = nn.Linear(128, 64)           # 第二个全连接层
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)# 输出层
        self.relu = nn.ReLU()                   # ReLU 激活函数
        self.sigmoid = nn.Sigmoid()             # Sigmoid 激活函数用于二分类

     def forward(self, x):
        x = self.relu(self.fc1(x))              # 第一层
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))           # 输出层
        return x                                 # 返回概率值

# 使用示例
input_size = 512  # 输入特征的维度
model = BinaryClassifier(input_size)

# 打印模型结构
print(model)

BinaryClassifier(
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [315]:
z = z.detach().numpy()
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(z, labels, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [316]:
import numpy as np
import random
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30
acc2=[]
for epoch in range(num_epochs):
    model.train()  # 设置为训练模式
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1).float())
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    model.eval()  # 设置为评估模式
    with torch.no_grad():  # 不计算梯度
        val_outputs = model(X_val)  # 在验证集上进行前向传播
        val_predicted = (val_outputs >= 0.5).float()  # 将输出转换为0或1的预测
        correct_predictions = (val_predicted.view(-1) == y_val).sum().item()  # 计算正确预测的数量
        val_accuracy = correct_predictions / y_val.size(0)  # 计算验证集的准确率
        acc2.append(val_accuracy)
    #if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

Epoch [1/30], Loss: 0.6945, Val Accuracy: 0.4797
Epoch [2/30], Loss: 0.6929, Val Accuracy: 0.4797
Epoch [3/30], Loss: 0.6910, Val Accuracy: 0.4797
Epoch [4/30], Loss: 0.6884, Val Accuracy: 0.4797
Epoch [5/30], Loss: 0.6854, Val Accuracy: 0.4797
Epoch [6/30], Loss: 0.6818, Val Accuracy: 0.4797
Epoch [7/30], Loss: 0.6774, Val Accuracy: 0.4715
Epoch [8/30], Loss: 0.6721, Val Accuracy: 0.5041
Epoch [9/30], Loss: 0.6657, Val Accuracy: 0.5366
Epoch [10/30], Loss: 0.6582, Val Accuracy: 0.5610
Epoch [11/30], Loss: 0.6495, Val Accuracy: 0.5610
Epoch [12/30], Loss: 0.6396, Val Accuracy: 0.5366
Epoch [13/30], Loss: 0.6282, Val Accuracy: 0.5041
Epoch [14/30], Loss: 0.6153, Val Accuracy: 0.4797
Epoch [15/30], Loss: 0.6003, Val Accuracy: 0.4634
Epoch [16/30], Loss: 0.5830, Val Accuracy: 0.4472
Epoch [17/30], Loss: 0.5633, Val Accuracy: 0.4472
Epoch [18/30], Loss: 0.5408, Val Accuracy: 0.4309
Epoch [19/30], Loss: 0.5157, Val Accuracy: 0.4472
Epoch [20/30], Loss: 0.4882, Val Accuracy: 0.4472
Epoch [21

In [317]:
from sklearn.metrics import f1_score
with torch.no_grad():
    val_outputs = model(X_val)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 42.28%
Validation F1 Score: 0.4228


In [318]:
with torch.no_grad():
    val_outputs = model(X_test)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 54.90%
Validation F1 Score: 0.5000


In [319]:
import torch
import torch.nn as nn

class FeatureFusion(nn.Module):
    def __init__(self, input_dim_u1, input_dim_u2):
        super(FeatureFusion, self).__init__()
        # 定义线性层以调整特征维度
        self.linear_u1 = nn.Linear(input_dim_u1, 512)
        self.linear_u2 = nn.Linear(input_dim_u2, 512)

    def forward(self, u1, u2):
        # 调整特征维度
        u1_transformed = self.linear_u1(u1)
        u2_transformed = self.linear_u2(u2)

        # 计算外积
        z = self.outer_product(u1_transformed, u2_transformed)
        return z

    def outer_product(self, x, y):
        return torch.einsum('ik,jk->ij', x, y)

# 示例输入
feature_dim_u1 = 75648  # 模态1特征维度
feature_dim_u2 = 56832  # 模态2特征维度
feature_fusion = FeatureFusion(feature_dim_u1, feature_dim_u2)

# 计算融合特征
z = feature_fusion(x, x1)

In [320]:
import torch
import torch.nn as nn

class BinaryClassifier(nn.Module):
     def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # 第一个全连接层
        self.fc2 = nn.Linear(128, 64)           # 第二个全连接层
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)# 输出层
        self.relu = nn.ReLU()                   # ReLU 激活函数
        self.sigmoid = nn.Sigmoid()             # Sigmoid 激活函数用于二分类

     def forward(self, x):
        x = self.relu(self.fc1(x))              # 第一层
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))           # 输出层
        return x                                 # 返回概率值

# 使用示例
input_size = 815  # 输入特征的维度
model = BinaryClassifier(input_size)

# 打印模型结构
print(model)

BinaryClassifier(
  (fc1): Linear(in_features=815, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [321]:
z = z.detach().numpy()
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(z, labels, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [322]:
import numpy as np
import random
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30
acc3=[]
for epoch in range(num_epochs):
    model.train()  # 设置为训练模式
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1).float())
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    model.eval()  # 设置为评估模式
    with torch.no_grad():  # 不计算梯度
        val_outputs = model(X_val)  # 在验证集上进行前向传播
        val_predicted = (val_outputs >= 0.5).float()  # 将输出转换为0或1的预测
        correct_predictions = (val_predicted.view(-1) == y_val).sum().item()  # 计算正确预测的数量
        val_accuracy = correct_predictions / y_val.size(0)  # 计算验证集的准确率
        acc3.append(val_accuracy)
    #if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}')

Epoch [1/30], Loss: 0.6955, Val Accuracy: 0.4390
Epoch [2/30], Loss: 0.6882, Val Accuracy: 0.4228
Epoch [3/30], Loss: 0.6830, Val Accuracy: 0.4553
Epoch [4/30], Loss: 0.6755, Val Accuracy: 0.4228
Epoch [5/30], Loss: 0.6668, Val Accuracy: 0.4634
Epoch [6/30], Loss: 0.6539, Val Accuracy: 0.5203
Epoch [7/30], Loss: 0.6405, Val Accuracy: 0.4228
Epoch [8/30], Loss: 0.6317, Val Accuracy: 0.4390
Epoch [9/30], Loss: 0.6103, Val Accuracy: 0.5122
Epoch [10/30], Loss: 0.6028, Val Accuracy: 0.4390
Epoch [11/30], Loss: 0.5811, Val Accuracy: 0.4390
Epoch [12/30], Loss: 0.5653, Val Accuracy: 0.4878
Epoch [13/30], Loss: 0.5455, Val Accuracy: 0.4715
Epoch [14/30], Loss: 0.5233, Val Accuracy: 0.4472
Epoch [15/30], Loss: 0.5056, Val Accuracy: 0.4553
Epoch [16/30], Loss: 0.4778, Val Accuracy: 0.5041
Epoch [17/30], Loss: 0.4599, Val Accuracy: 0.4472
Epoch [18/30], Loss: 0.4302, Val Accuracy: 0.4634
Epoch [19/30], Loss: 0.4014, Val Accuracy: 0.4959
Epoch [20/30], Loss: 0.3817, Val Accuracy: 0.4228
Epoch [21

In [323]:
from sklearn.metrics import f1_score
with torch.no_grad():
    val_outputs = model(X_val)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_val).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_val_np = y_val.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_val_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 47.97%
Validation F1 Score: 0.5224


In [324]:
with torch.no_grad():
    val_outputs = model(X_test)
    val_predictions = (val_outputs >= 0.5).float()  # 预测类别
    val_accuracy = (val_predictions.view(-1) == y_test).float().mean()  # 计算准确率
    print(f'Validation Accuracy: {val_accuracy.item() * 100:.2f}%')
    val_predictions_np = val_predictions.cpu().numpy().flatten()
    y_test_np = y_test.cpu().numpy().flatten()
    # 计算 F1 分数
    val_f1 = f1_score(y_test_np, val_predictions_np)
    print(f'Validation F1 Score: {val_f1:.4f}')

Validation Accuracy: 50.00%
Validation F1 Score: 0.5526


In [139]:
import pandas as pd
loss_df = pd.DataFrame({
    "Accuracy_fusion1": acc,
    'Acc2':acc1,
    'Acc3':acc2,
    'Acc4':acc3
})
loss_df.to_csv("Accuracy.csv", index=False)